# Pt 1: Get the Weather Description and Amount of Precipitation for Each City

In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import requests

from citipy import citipy
from config import weather_api_key
from datetime import datetime

In [2]:
# Create a set of 1,500 random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lats_lngs = zip(lats, lngs)
lats_lngs

In [3]:
# Add latitudes and longitudes to a list
coordinates = list(lats_lngs)
coordinates

[(68.36766306731997, 105.43105734959806),
 (4.8164741237257545, -24.78935868253967),
 (-67.66647772322223, 118.09230301796617),
 (11.808694672101396, 37.007760397091715),
 (10.498901951609483, 174.9715619139023),
 (-89.1676682856133, 112.96614310342886),
 (53.974328825629186, 159.98727036641287),
 (-12.921872642272461, 47.357097417051364),
 (-2.7323464601461467, 82.65024561485575),
 (45.35700810778292, -179.79534858928716),
 (23.652528382158295, -143.38194063675704),
 (-59.5694449635309, -169.91735746018344),
 (46.06710226489264, -91.50572030468616),
 (-62.1159486691829, 80.45446802535696),
 (-86.91709508459245, -29.039508716157712),
 (-19.045909502659512, 80.20189228030466),
 (3.7333508585285244, -9.315401521318904),
 (60.94110673840572, -139.54497452303235),
 (-42.52990309564581, -3.129344593371627),
 (-0.7459141724512648, 146.76917030304622),
 (81.13103566057518, -16.47907244147649),
 (-10.119962229023784, -119.46088118076511),
 (7.742013514252903, 81.65894487257651),
 (28.213056310

In [41]:
# Get the nearest city using the citipy module
cities = []

# Identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique then add it to the cities list
    if city not in cities:
        cities.append(city)

# Check count to confirm sufficient cities
len(cities)

602

In [42]:
# Build URL for Weather API call using my weather_api_key
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [52]:
# Test if URL works
test_city_url = url + "&q=" + "Oklahoma City"
test_city_weather = requests.get(test_city_url)
test_city_weather

<Response [200]>

In [53]:
# Get the JSON text of the "get" request
test_city_weather.json()

{'coord': {'lon': -97.52, 'lat': 35.47},
 'weather': [{'id': 500,
   'main': 'Rain',
   'description': 'light rain',
   'icon': '10d'}],
 'base': 'stations',
 'main': {'temp': 49.35,
  'feels_like': 41.9,
  'temp_min': 48.2,
  'temp_max': 51.01,
  'pressure': 1015,
  'humidity': 78},
 'visibility': 14484,
 'wind': {'speed': 10.29, 'deg': 160},
 'rain': {'1h': 0.25},
 'clouds': {'all': 20},
 'dt': 1582480053,
 'sys': {'type': 1,
  'id': 4424,
  'country': 'US',
  'sunrise': 1582463289,
  'sunset': 1582503551},
 'timezone': -21600,
 'id': 4544349,
 'name': 'Oklahoma City',
 'cod': 200}

In [64]:
# Create an empty list to hold the weather data
weather_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in the cities list
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        
    # Create endpoint URL with each city
    city_url = url + "&q=" + city
    
    # Log the URL, record, and set numbers and the city
    print(f"Processing record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count
    record_count += 1

# Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_description = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        
        # Convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Get rain and/or snow data if it is raining and/or snowing in the cities
        try:
            city_rain = city_weather["rain"]["3h"]
        except:
            city_rain = 0   
        try:
            city_snow = city_weather["snow"]["3h"]
        except:
            city_snow = 0

        # Append city weather information into weather_data list
        weather_data.append({"City": city.title(), 
                             "Country": city_country,
                             "Date": city_date,
                             "Lat": city_lat,
                             "Lng": city_lng,
                             "Max Temp": city_max_temp,
                             "Humidity": city_humidity,
                             "Cloudiness": city_clouds,
                             "Wind Speed": city_wind,
                             "Current Description": city_weather_description,
                             "Rain inches (last 3 hrs)": city_rain,
                             "Snow inches (last 3 hrs)": city_snow})        

# If an error is experience, skip the city
    except:
        print("City not found. Skipping...")
        pass       
        
# Indicate that Data Retrieval is complete.
print("--------------------------------")
print("Data Retrieval Complete.        ")
print("--------------------------------")

Beginning Data Retrieval     
-----------------------------
Processing record 1 of Set 1 | khatanga
0.25
Processing record 2 of Set 1 | sao filipe
Processing record 3 of Set 1 | albany
Processing record 4 of Set 1 | bahir dar
Processing record 5 of Set 1 | butaritari
6.37
Processing record 6 of Set 1 | milkovo
0.19
Processing record 7 of Set 1 | ambanja
0.19
Processing record 8 of Set 1 | matara
Processing record 9 of Set 1 | provideniya
Processing record 10 of Set 1 | hilo
Processing record 11 of Set 1 | vaini
Processing record 12 of Set 1 | superior
Processing record 13 of Set 1 | busselton
0.69
Processing record 14 of Set 1 | ushuaia
Processing record 15 of Set 1 | hithadhoo
2.13
Processing record 16 of Set 1 | harper
2.75
Processing record 17 of Set 1 | haines junction
0.94
Processing record 18 of Set 1 | saldanha
Processing record 19 of Set 1 | lorengau
Processing record 20 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing record 21 of Set 1 | atuona
0.5
Processin

2.94
Processing record 37 of Set 4 | morlaix
Processing record 38 of Set 4 | dikson
1.75
Processing record 39 of Set 4 | laguna
Processing record 40 of Set 4 | egvekinot
Processing record 41 of Set 4 | dumai
Processing record 42 of Set 4 | attawapiskat
City not found. Skipping...
Processing record 43 of Set 4 | dubrovnik
Processing record 44 of Set 4 | udachnyy
Processing record 45 of Set 4 | sur
Processing record 46 of Set 4 | avarua
Processing record 47 of Set 4 | ugoofaaru
Processing record 48 of Set 4 | manali
Processing record 49 of Set 4 | belushya guba
City not found. Skipping...
Processing record 50 of Set 4 | kisangani
Processing record 1 of Set 5 | broken hill
Processing record 2 of Set 5 | margate
Processing record 3 of Set 5 | saint-joseph
Processing record 4 of Set 5 | evensk
0.25
Processing record 5 of Set 5 | half moon bay
Processing record 6 of Set 5 | potiskum
Processing record 7 of Set 5 | corbeanca
Processing record 8 of Set 5 | chifeng
Processing record 9 of Set 5 |

Processing record 26 of Set 8 | el wasta
City not found. Skipping...
Processing record 27 of Set 8 | yulara
Processing record 28 of Set 8 | palana
Processing record 29 of Set 8 | isangel
Processing record 30 of Set 8 | linxia
Processing record 31 of Set 8 | porterville
Processing record 32 of Set 8 | miracema do tocantins
1.19
Processing record 33 of Set 8 | salalah
Processing record 34 of Set 8 | borba
1.62
Processing record 35 of Set 8 | alfredo chaves
Processing record 36 of Set 8 | lagoa
Processing record 37 of Set 8 | tazovskiy
Processing record 38 of Set 8 | gorontalo
0.13
Processing record 39 of Set 8 | carauari
0.75
Processing record 40 of Set 8 | sinjai
Processing record 41 of Set 8 | paradwip
City not found. Skipping...
Processing record 42 of Set 8 | lewiston
Processing record 43 of Set 8 | shebalino
0.19
Processing record 44 of Set 8 | kirya
Processing record 45 of Set 8 | sokoni
Processing record 46 of Set 8 | yumen
Processing record 47 of Set 8 | gravdal
Processing record

Processing record 10 of Set 12 | hazorasp
Processing record 11 of Set 12 | kimberley
Processing record 12 of Set 12 | burnie
Processing record 13 of Set 12 | quang ngai
0.19
Processing record 14 of Set 12 | aswan
Processing record 15 of Set 12 | jingdezhen
Processing record 16 of Set 12 | beloha
2.63
Processing record 17 of Set 12 | masindi
Processing record 18 of Set 12 | barabai
2.13
Processing record 19 of Set 12 | newry
Processing record 20 of Set 12 | balkanabat
Processing record 21 of Set 12 | bubaque
Processing record 22 of Set 12 | povorino
2.56
Processing record 23 of Set 12 | gardez
Processing record 24 of Set 12 | mackay
Processing record 25 of Set 12 | weyburn
Processing record 26 of Set 12 | deyang
Processing record 27 of Set 12 | vsevolodo-vilva
City not found. Skipping...
Processing record 28 of Set 12 | orlik
Processing record 29 of Set 12 | thurso
Processing record 30 of Set 12 | emba
Processing record 31 of Set 12 | almenara
0.75
Processing record 32 of Set 12 | kishi

In [65]:
len(weather_data)

555

In [75]:
# Convert the array of dictionaries to a Pandas DataFrame
weather_data_df = pd.DataFrame(weather_data)
weather_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,Khatanga,RU,2020-02-23 18:27:20,71.97,102.50,12.67,92,98,15.46,light snow,0.00,0.25
1,Sao Filipe,CV,2020-02-23 18:27:20,14.90,-24.50,68.52,83,100,18.84,overcast clouds,0.00,0.00
2,Albany,US,2020-02-23 18:27:21,42.60,-73.97,51.01,29,75,4.70,broken clouds,0.00,0.00
3,Bahir Dar,ET,2020-02-23 18:26:17,11.59,37.39,75.31,25,35,7.09,scattered clouds,0.00,0.00
4,Butaritari,KI,2020-02-23 18:25:37,3.07,172.79,84.70,76,100,17.56,moderate rain,6.37,0.00
5,Milkovo,RU,2020-02-23 18:21:05,54.72,158.62,-4.72,99,100,0.45,light snow,0.00,0.19
6,Ambanja,MG,2020-02-23 18:27:21,-13.68,48.45,78.33,84,100,4.65,light rain,0.19,0.00
7,Matara,LK,2020-02-23 18:27:21,5.95,80.54,75.51,87,94,10.89,overcast clouds,0.00,0.00
8,Provideniya,RU,2020-02-23 18:27:22,64.38,-173.30,-9.76,73,0,2.24,clear sky,0.00,0.00
9,Hilo,US,2020-02-23 18:27:22,19.73,-155.09,70.00,64,40,4.70,scattered clouds,0.00,0.00


In [92]:
# Create the output file (CSV)
output_data_file = "challenge_data/WeatherPy_challenge.csv"

# Export the weather_data DataFrame into a CSV file
weather_data_df.to_csv(output_data_file, index_label="City_ID")

In [115]:
#Answer this question using Pandas methods: How many cities have recorded rainfall or snow?

# Cities that recorded rain
cities_rain = weather_data_df[(weather_data_df["Rain inches (last 3 hrs)"] > 0)].count()
cities_rain_count = cities_rain["Rain inches (last 3 hrs)"]

# Cities that recorded snow
cities_snow = weather_data_df[(weather_data_df["Snow inches (last 3 hrs)"] > 0)].count()
cities_snow_count = cities_snow["Snow inches (last 3 hrs)"]

print(f"There were {cities_rain_count} cities that recorded rain.")
print(f"There were {city_snow_count} cities that recorded snow.")

There were 57 cities that recorded rain.
There were 20 cities that recorded snow.
